# Diagonal Coulomb operators and double-factorized Trotter simulation

In this tutorial, we show how to use ffsim to simulate diagonal Coulomb operators and approximate time evolution by a molecular Hamiltonian in the double-factorized representation.

## Double-factorized representation of the molecular Hamiltonian

The molecular Hamiltonian is

$$
    H = \sum_{\sigma, pq} h_{pq} a^\dagger_{\sigma, p} a_{\sigma, q}
        + \frac12 \sum_{\sigma \tau, pqrs} h_{pqrs}
        a^\dagger_{\sigma, p} a^\dagger_{\tau, r} a_{\tau, s} a_{\sigma, q}
        + \text{constant}.
$$

This representation of the Hamiltonian is daunting for quantum simulations because the number of terms scales as $N^4$ where $N$ is the number of spatial orbitals. An alternative representation can be obtained by performing a "double-factorization" of the two-body tensor $h_{pqrs}$:

$$
    H = \sum_{\sigma, pq} h'_{pq} a^\dagger_{\sigma, p} a_{\sigma, q}
    + \sum_{k=1}^L \mathcal{W}_k \mathcal{J}_k \mathcal{W}_k^\dagger
    + \text{constant}'.
$$

Here each $\mathcal{W}_k$ is an [orbital rotation](./02-orbital-rotation.ipynb) and each $\mathcal{J}_k$ is a so-called diagonal Coulomb operator, which is an operator of the form

$$
    \mathcal{J} = \frac12\sum_{\sigma \tau, ij} \mathbf{J}_{ij} n_{\sigma, i} n_{\tau, j},
$$

where $n_{\sigma, i} = a^\dagger_{\sigma, i} a_{\sigma, i}$ is the occupation number operator and $\mathbf{J}_{ij}$ is a real symmetric matrix.

In the cell below, we construct the Hamiltonian for an ethene molecule at a stretched bond length and then get the double-factorized representation of the Hamiltonian.

In [1]:
import pyscf
import ffsim

# Build a stretched ethene molecule
bond_distance = 2.678
a = 0.5 * bond_distance
b = a + 0.5626
c = 0.9289
mol = pyscf.gto.Mole()
mol.build(
    atom=[
        ["C", (0, 0, a)],
        ["C", (0, 0, -a)],
        ["H", (0, c, b)],
        ["H", (0, -c, b)],
        ["H", (0, c, -b)],
        ["H", (0, -c, -b)],
    ],
    basis="sto-6g",
    symmetry="d2h",
)

# Define active space
active_space = range(mol.nelectron // 2 - 2, mol.nelectron // 2 + 2)

# Get molecular data and molecular Hamiltonian (one- and two-body tensors)
mol_data = ffsim.MolecularData.from_mole(mol, active_space=active_space)
norb = mol_data.norb
nelec = mol_data.nelec
mol_hamiltonian = mol_data.hamiltonian

# Get the Hamiltonian in the double-factorized representation
df_hamiltonian = ffsim.DoubleFactorizedHamiltonian.from_molecular_hamiltonian(
    mol_hamiltonian
)

converged SCF energy = -77.4456267643962


Here, `mol_hamiltonian` is an instance of `MolecularHamiltonian`, a dataclass that stores the one- and two-body tensors, and `df_hamiltonian` is an instance of `DoubleFactorizedHamiltonian`, a dataclass that stores the updated one-body-tensor, diagonal Coulomb matrices, and orbital rotations. In the cell below, we print out the shapes of the tensors describing the original and double-factorized representations.

In [2]:
print("Original representation")
print("-----------------------")
print("One-body tensor shape:")
print(mol_hamiltonian.one_body_tensor.shape)
print()
print("Two-body tensor shape:")
print(mol_hamiltonian.two_body_tensor.shape)
print()

print("Double-factorized representation")
print("--------------------------------")
print("One-body tensor shape:")
print(df_hamiltonian.one_body_tensor.shape)
print()
print("Diagonal Coulomb matrices shape:")
print(df_hamiltonian.diag_coulomb_mats.shape)
print()
print("Orbital rotations shape:")
print(df_hamiltonian.orbital_rotations.shape)

Original representation
-----------------------
One-body tensor shape:
(4, 4)

Two-body tensor shape:
(4, 4, 4, 4)

Double-factorized representation
--------------------------------
One-body tensor shape:
(4, 4)

Diagonal Coulomb matrices shape:
(10, 4, 4)

Orbital rotations shape:
(10, 4, 4)


## Trotter simulation of the double-factorized Hamiltonian

In the rest of this tutorial, we'll show how to use ffsim to implement time evolution of the double-factorized Hamiltonian via Trotter-Suzuki formulas. Although ffsim already has this functionality built-in, we will first manually implement a first-order asymmetric product formula to demonstrate the use of ffsim's basic operations.

### Brief background on Trotter-Suzuki formulas
Trotter-Suzuki formulas are used to approximate time evolution by a Hamiltonian $H$ which is decomposed as a sum of terms:

$$
H = \sum_k H_k.
$$

Time evolution by time $t$ is given by the unitary operator

$$
e^{i H t}.
$$

To approximate this operator, the total evolution time is first divided into a number of smaller time steps, called "Trotter steps":

$$
e^{i H t} = (e^{i H t / r})^r.
$$

The time evolution for a single Trotter step is then approximated using a product formula, which approximates the exponential of a sum of terms by a product of exponentials of the individual terms. The formulas are approximate because the terms do not in general commute. A first-order asymmetric product formula has the form

$$
e^{i H \tau} \approx \prod_k e^{i H_k \tau}.
$$

Higher-order formulas can be derived which yield better approximations.

### Implementing Trotter simulation of the double-factorized Hamiltonian

First, we'll write a function to simulate a single Trotter step of the Hamiltonian. Recall the form of the Hamiltonian (ignoring the additive constant):

$$
    H = \sum_{\sigma, pq} h'_{pq} a^\dagger_{\sigma, p} a_{\sigma, q}
    + \sum_{k=1}^L \mathcal{W}_k \mathcal{J}_k \mathcal{W}_k^\dagger
$$

We think of this Hamiltonian as composed of $L + 1$ terms: the one-body term, which is a quadratic Hamiltonian, and the $L$ "rotated diagonal Coulomb operators." As described in [this tutorial](./02-orbital-rotation.ipynb), time evolution by the quadratic Hamiltonian can be implemented using the `apply_num_op_sum_evolution` function. Similarly, time evolution by a rotated diagonal coulomb operator can be implemented using the `apply_diag_coulomb_evolution` function.

In [3]:
import numpy as np


def simulate_trotter_step_double_factorized(
    vec: np.ndarray,
    hamiltonian: ffsim.DoubleFactorizedHamiltonian,
    time: float,
    norb: int,
    nelec: tuple[int, int],
) -> np.ndarray:
    # Diagonalize the one-body term
    one_body_energies, one_body_basis_change = np.linalg.eigh(
        hamiltonian.one_body_tensor
    )
    # Simulate the one-body term
    vec = ffsim.apply_num_op_sum_evolution(
        vec,
        one_body_energies,
        time,
        norb=norb,
        nelec=nelec,
        orbital_rotation=one_body_basis_change,
    )
    # Simulate the two-body terms
    for diag_coulomb_mat, orbital_rotation in zip(
        hamiltonian.diag_coulomb_mats, hamiltonian.orbital_rotations
    ):
        vec = ffsim.apply_diag_coulomb_evolution(
            vec,
            diag_coulomb_mat,
            time,
            norb=norb,
            nelec=nelec,
            orbital_rotation=orbital_rotation,
        )
    return vec

We finish by writing a higher-level function that handles splitting the total time evolution into multiple Trotter steps, simulating each Trotter step using the function we just wrote.

In [4]:
def simulate_trotter_double_factorized(
    vec: np.ndarray,
    hamiltonian: ffsim.DoubleFactorizedHamiltonian,
    time: float,
    norb: int,
    nelec: tuple[int, int],
    n_steps: int = 1,
) -> np.ndarray:
    step_time = time / n_steps
    for _ in range(n_steps):
        vec = simulate_trotter_step_double_factorized(
            vec,
            hamiltonian,
            step_time,
            norb=norb,
            nelec=nelec,
        )
    return vec

To test our implementation, we'll apply the time evolution to the Hartree-Fock state, which is a Slater determinant with electrons occupying the lowest-energy molecular orbitals. In the following code cell, we'll create this state and calculate its energy. It should match the value output by pySCF when we first created the molecule. To calculate the energy, we convert the Hamiltonian to a SciPy `LinearOperator`.

In [5]:
# Construct the Hartree-Fock state
initial_state = ffsim.hartree_fock_state(norb, nelec)

# Get the Hamiltonian as a LinearOperator
hamiltonian = ffsim.linear_operator(mol_hamiltonian, norb=norb, nelec=nelec)

# Check the energy ⟨ψ|H|ψ⟩ of the Hartree-Fock state
hf_energy = np.real(np.vdot(initial_state, hamiltonian @ initial_state))
print(f"Hartree Fock energy: {hf_energy}")

Hartree Fock energy: -77.44562676439628


Now, we set the evolution time and calculate the exact result of time evolution by directly exponentiating the Hamiltonian using SciPy. Later, we will compare the result of our approximate time evolution with this exact result.

In [6]:
import scipy.sparse.linalg

time = 5.0

exact_state = scipy.sparse.linalg.expm_multiply(
    -1j * time * hamiltonian,
    initial_state,
    traceA=ffsim.trace(mol_hamiltonian, norb=norb, nelec=nelec),
)

fidelity = abs(np.vdot(exact_state, initial_state))
print(f"Fidelity of evolved state w.r.t. initial state: {fidelity}")

Fidelity of evolved state w.r.t. initial state: 0.9315062301399599


Now, let's test our implementation.

In [7]:
final_state = simulate_trotter_double_factorized(
    initial_state,
    df_hamiltonian,
    time,
    norb=norb,
    nelec=nelec,
    n_steps=1,
)

fidelity = abs(np.vdot(final_state, exact_state))
print(f"Fidelity of Trotter-evolved state with exact state: {fidelity}")

Fidelity of Trotter-evolved state with exact state: 0.9928527668213422


The fidelity of the final result can be improved by increasing the number of Trotter steps.

In [8]:
final_state = simulate_trotter_double_factorized(
    initial_state,
    df_hamiltonian,
    time,
    norb=norb,
    nelec=nelec,
    n_steps=10,
)

fidelity = abs(np.vdot(final_state, exact_state))
print(f"Fidelity of Trotter-evolved state with exact state: {fidelity}")

Fidelity of Trotter-evolved state with exact state: 0.9999320851286914


As mentioned above, ffsim already includes functionality for Trotter simulation of double-factorized Hamiltonians. The implementation in ffsim includes higher-order Trotter-Suzuki formulas. The first-order asymmetric formula that we just implemented corresponds to `order=0` in ffsim's implementation. `order=1` corresponds to the first-order symmetric (commonly known as the second-order) formula, `order=2` corresponds to the second-order symmetric (fourth-order) formula, and so on.

In the code cell below, we reproduce the results of our manually implemented function using ffsim's built-in implementation.

In [9]:
final_state = ffsim.simulate_trotter_double_factorized(
    initial_state,
    df_hamiltonian,
    time,
    norb=norb,
    nelec=nelec,
    n_steps=10,
    order=0,
)

fidelity = abs(np.vdot(final_state, exact_state))
print(f"Fidelity of Trotter-evolved state with exact state: {fidelity}")

Fidelity of Trotter-evolved state with exact state: 0.9999320851286914


A higher order formula achieves a higher fidelity with fewer Trotter steps:

In [10]:
final_state = ffsim.simulate_trotter_double_factorized(
    initial_state,
    df_hamiltonian,
    time,
    norb=norb,
    nelec=nelec,
    n_steps=3,
    order=1,
)

fidelity = abs(np.vdot(final_state, exact_state))
print(f"Fidelity of Trotter-evolved state with exact state: {fidelity}")

Fidelity of Trotter-evolved state with exact state: 0.9999913261307772
